"Speed Dating" data analysis
-----
My first aim is to approach the data on a non-supervised way so we'll start with some plots.

In [1]:
import numpy as np
import pandas as pd

In [11]:
speedDatingDF = pd.read_csv("Speed Dating Data.csv",encoding = "ISO-8859-1")
speedDatingDF.dtypes

iid           int64
id          float64
gender        int64
idg           int64
condtn        int64
wave          int64
round         int64
position      int64
positin1    float64
order         int64
partner       int64
pid         float64
match         int64
int_corr    float64
samerace      int64
age_o       float64
race_o      float64
pf_o_att    float64
pf_o_sin    float64
pf_o_int    float64
pf_o_fun    float64
pf_o_amb    float64
pf_o_sha    float64
dec_o         int64
attr_o      float64
sinc_o      float64
intel_o     float64
fun_o       float64
amb_o       float64
shar_o      float64
             ...   
amb1_3      float64
shar1_3     float64
attr7_3     float64
sinc7_3     float64
intel7_3    float64
fun7_3      float64
amb7_3      float64
shar7_3     float64
attr4_3     float64
sinc4_3     float64
intel4_3    float64
fun4_3      float64
amb4_3      float64
shar4_3     float64
attr2_3     float64
sinc2_3     float64
intel2_3    float64
fun2_3      float64
amb2_3      float64


I have way too much characteristics so now I must find a way to break through all this data into some conclusion. The question we want to answer is:

"Are these two people going to match?"

In [13]:
speedDatingDF.groupby('wave')

<bound method GroupBy.mean of <pandas.core.groupby.DataFrameGroupBy object at 0x1065d5f60>>